<a href="https://colab.research.google.com/github/caltunay/dd_competition_eq_damage_prediction/blob/master/03_09_retake_eq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
import numpy as np

In [0]:
test_vals = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Earthquake Damage Prediction/test_values.csv'

train_vals = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Earthquake Damage Prediction/train_values.csv'
train_labs = '/content/drive/My Drive/Colab Notebooks/Data/Competition - DrivenData - Earthquake Damage Prediction/train_labels.csv'

In [0]:
all = pd.merge(left = pd.read_csv(train_vals), right = pd.read_csv(train_labs), on = 'building_id', how = 'inner')
all.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,2
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0,3


In [0]:
categorik = all.select_dtypes('object').columns.tolist()

all_dummied = pd.get_dummies(data = all, prefix_sep = '|', columns = categorik, drop_first = True)

In [0]:
damage_dict = {1:0, 2:1, 3:2}

all_dummied['damage_grade'] = all_dummied['damage_grade'].replace(damage_dict)

In [0]:
# dicts for target encoding
geo1_dict = all_dummied.groupby('geo_level_1_id').mean()['damage_grade'].to_dict()
geo2_dict = all_dummied.groupby('geo_level_2_id').mean()['damage_grade'].to_dict()
geo3_dict = all_dummied.groupby('geo_level_3_id').mean()['damage_grade'].to_dict()

In [0]:
all_dummied['geo_level_1_id'] = all_dummied['geo_level_1_id'].replace(geo1_dict)
all_dummied['geo_level_2_id'] = all_dummied['geo_level_2_id'].replace(geo2_dict)
all_dummied['geo_level_3_id'] = all_dummied['geo_level_3_id'].replace(geo3_dict)

In [0]:
numerik = all_dummied.drop(labels = 'building_id geo_level_1_id geo_level_2_id geo_level_3_id damage_grade'.split(), axis = 1).nunique().sort_values(ascending = False).index[:5].tolist()

In [0]:
all_dummied[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
age,260601.0,26.535029,73.565937,0.0,10.0,15.0,30.0,995.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
age995 = all_dummied[all_dummied['age'] == 995]
remainder = all_dummied[all_dummied['age'] != 995]

In [0]:
age_dict = remainder['age'].value_counts(normalize = True).to_dict()

In [0]:
el = []

for k, v in age_dict.items():
    
    repetition = round(v * 1390)
    temp_l = [k] * repetition
    
    el = el + temp_l

# shuffle(el)

In [0]:
from random import shuffle

In [0]:
# shuffle(el)
len(el)

1392

In [0]:
el.pop()
el.pop()

200

In [0]:
age995['age'] = el

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
all_clean = pd.concat([remainder, age995], axis = 0).reset_index(drop = True)

In [0]:
all_clean[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
age,260601.0,21.340766,19.603808,0.0,10.0,15.0,30.0,200.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
age_series = pd.cut(x = all_clean['age'], bins = [0,10,30,80,200], include_lowest = True, labels = 'new young old ancient'.split()).rename('age_bins', inplace = True)

age_series

In [0]:
all_clean = pd.concat([all_clean, age_series], axis = 1).drop(labels = 'age', axis = 1)

In [0]:
numerik.pop(numerik.index('age'))

'age'

In [0]:
all_clean[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
area_percentage,260601.0,8.018051,4.392231,1.0,5.0,7.0,9.0,100.0
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
series_area = pd.cut(x = all_clean['area_percentage'], bins = [1,5,9,100], labels = 'small average big'.split(), include_lowest = True).rename('area_bins')

all_clean = pd.concat([all_clean, series_area], axis = 1).drop(labels = 'area_percentage', axis = 1)

In [0]:
all_clean.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w,age_bins,area_bins
0,802906,1.161724,1.740741,1.837838,2,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,young,average
1,28830,1.485273,1.487437,1.062500,2,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,new,average
2,94947,1.563369,1.518750,1.580882,2,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,new,small
3,590882,1.000960,1.107317,1.096774,2,5,0,1,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,new,average
4,201944,1.337713,1.348748,1.368852,3,9,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,young,average


In [0]:
numerik.pop(numerik.index('area_percentage'))

'area_percentage'

In [0]:
all_clean[numerik].describe().T

,count,mean,std,min,25%,50%,75%,max
height_percentage,260601.0,5.434365,1.918418,2.0,4.0,5.0,6.0,32.0
count_families,260601.0,0.983949,0.418389,0.0,1.0,1.0,1.0,9.0
count_floors_pre_eq,260601.0,2.129723,0.727665,1.0,2.0,2.0,2.0,9.0


In [0]:
all_clean['height_percentage'] = np.log1p(all_clean['height_percentage'])
all_clean['count_floors_pre_eq'] = np.log1p(all_clean['count_floors_pre_eq'])
all_clean['count_families'] = np.log1p(all_clean['count_families'])

In [0]:
all_clean = pd.get_dummies(data = all_clean, columns = ['age_bins', 'area_bins'], drop_first = True, prefix_sep = '|')

In [0]:
print(all_clean.shape)
all_clean.head()

(260601, 65)


,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w,age_bins|young,age_bins|old,age_bins|ancient,area_bins|average,area_bins|big
0,802906,1.161724,1.740741,1.837838,1.098612,1.791759,1,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
1,28830,1.485273,1.487437,1.062500,1.098612,2.079442,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,94947,1.563369,1.518750,1.580882,1.098612,1.791759,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,590882,1.000960,1.107317,1.096774,1.098612,1.791759,0,1,0,0,0,0,1,1,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,201944,1.337713,1.348748,1.368852,1.386294,2.302585,1,0,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0


In [0]:
# will preprocess test set as well

In [0]:
test = pd.read_csv(test_vals)
test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


In [0]:
test_dummied = pd.get_dummies(data = test, columns = categorik, drop_first = True, prefix_sep = '|')

In [0]:
test995 = test_dummied[test_dummied['age'] == 995]
testremainder = test_dummied[test_dummied['age'] != 995]

In [0]:
el2 =[]

for k,v in age_dict.items():
    
    repetition = round(test995.shape[0] * v)
    temp_list = repetition * [k]
    
    el2 = el2 + temp_list
    
el2

In [0]:
len(el2)

459

In [0]:
el2 = el2 + [10, 10]

el2

In [0]:
shuffle(el2)

In [0]:
test995['age'] = el2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
test_clean = pd.concat([testremainder, test995], axis = 0).reset_index(drop = True)

In [0]:
test_clean['geo_level_1_id'] = test_clean['geo_level_1_id'].replace(geo1_dict)
test_clean['geo_level_2_id'] = test_clean['geo_level_2_id'].replace(geo2_dict)
test_clean['geo_level_3_id'] = test_clean['geo_level_3_id'].replace(geo3_dict)

In [0]:
test_clean[test_clean.geo_level_2_id == 0].shape

(4, 61)

In [0]:
geo2_mean = all_clean['geo_level_2_id'].mean()
geo3_mean = all_clean['geo_level_3_id'].mean()

In [0]:
test_clean['geo_level_2_id'] = test_clean['geo_level_2_id'].replace({0:geo2_mean})
test_clean[test_clean['geo_level_2_id'] == 0]

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w


In [0]:
test_clean['geo_level_3_id'] = test_clean['geo_level_3_id'].replace({0:geo3_mean})

In [0]:
age_series_test = pd.cut(x = test_clean['age'], bins = [0,10,30,80,200], include_lowest = True, labels = 'new young old ancient'.split()).rename('age_bins')

age_series_test

In [0]:
test_clean = pd.concat([test_clean, age_series_test], axis = 1)

In [0]:
series_area_test = pd.cut(x = test_clean['area_percentage'], bins = [1,5,9,100], labels = 'small average big'.split(), include_lowest = True).rename('area_bins')

test_clean = pd.concat([test_clean, series_area_test], axis = 1).drop(labels = 'area_percentage', axis = 1)

In [0]:
test_clean['height_percentage'] = np.log1p(test_clean['height_percentage'])
test_clean['count_floors_pre_eq'] = np.log1p(test_clean['count_floors_pre_eq'])
test_clean['count_families'] = np.log1p(test_clean['count_families'])

In [0]:
test_clean = pd.get_dummies(data = test_clean, columns = ['age_bins', 'area_bins'], prefix_sep = '|', drop_first = True)

In [0]:
test_clean = test_clean.drop(labels = 'age', axis = 1)

In [0]:
test_clean.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition|o,land_surface_condition|t,foundation_type|i,foundation_type|r,foundation_type|u,foundation_type|w,roof_type|q,roof_type|x,ground_floor_type|m,ground_floor_type|v,ground_floor_type|x,ground_floor_type|z,other_floor_type|q,other_floor_type|s,other_floor_type|x,position|o,position|s,position|t,plan_configuration|c,plan_configuration|d,plan_configuration|f,plan_configuration|m,plan_configuration|n,plan_configuration|o,plan_configuration|q,plan_configuration|s,plan_configuration|u,legal_ownership_status|r,legal_ownership_status|v,legal_ownership_status|w,age_bins|young,age_bins|old,age_bins|ancient,area_bins|average,area_bins|big
0,300051,1.794480,1.705036,1.631579,1.386294,1.945910,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
1,99355,1.161724,1.180851,1.000000,1.098612,1.791759,0,1,0,0,0,0,0,0,0,0,0,0.693147,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1
2,890251,1.000960,1.175000,2.000000,1.098612,1.791759,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,745817,0.730887,0.532194,0.437788,0.693147,1.386294,0,0,0,0,0,1,0,0,0,0,0,1.098612,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,421793,1.794480,1.689119,1.823529,1.386294,2.079442,0,1,0,0,0,0,0,0,0,0,0,0.693147,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0


In [0]:
s = 0

for tr, te in zip(all_clean.drop(labels ='damage_grade', axis = 1), test_clean.columns.tolist()):
    if tr == te:
        s = 0
    else:
        s += 1
s

0

In [0]:
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [0]:
x = all_clean.drop(labels = ['building_id', 'damage_grade'], axis = 1)
y = all_clean['damage_grade']
y_keras = pd.get_dummies(all_clean[['damage_grade']], columns = ['damage_grade'], prefix_sep = '|')

In [0]:
## Upsampled tryout

from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
sm = SMOTE(random_state = 42)
x_resampled, y_resampled = sm.fit_resample(x,y)

In [0]:
print(x_resampled.shape, y_resampled.shape)
print(y_resampled.value_counts())

In [0]:
x.shape

(260601, 63)

In [0]:
model = Sequential()

model.add(Dense(output_dim = 60, input_dim = 63, activation = 'relu', init = 'uniform'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu',))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 60, activation = 'relu'))
model.add(Dense(output_dim = 3, activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(x,y_keras, batch_size = 13030, epochs = 150, validation_split = .2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=63, activation="relu", units=60, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60, kernel_initializer="uniform")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=60, kernel_initializer="uniform")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` ca

Train on 208480 samples, validate on 52121 samples
Epoch 1/150
208480/208480 [==============================] - 2s 9us/step - loss: 1.0780 - acc: 0.5688 - val_loss: 1.0475 - val_acc: 0.5691
Epoch 2/150
208480/208480 [==============================] - 1s 4us/step - loss: 0.9843 - acc: 0.5689 - val_loss: 0.9324 - val_acc: 0.5691
Epoch 3/150
208480/208480 [==============================] - 1s 4us/step - loss: 0.9078 - acc: 0.5689 - val_loss: 0.8966 - val_acc: 0.5691
Epoch 4/150
208480/208480 [==============================] - 1s 4us/step - loss: 0.8842 - acc: 0.5689 - val_loss: 0.8730 - val_acc: 0.5691
Epoch 5/150
208480/208480 [==============================] - 1s 4us/step - loss: 0.8568 - acc: 0.5689 - val_loss: 0.8479 - val_acc: 0.5691
Epoch 6/150
208480/208480 [==============================] - 1s 4us/step - loss: 0.8280 - acc: 0.5776 - val_loss: 0.8065 - val_acc: 0.6441
Epoch 7/150
208480/208480 [==============================] - 1s 4us/step - loss: 0.7623 - acc: 0.6774 - val_loss: 0

In [0]:
submis_x = test_clean.drop(labels = 'building_id', axis = 1)

submis_df = pd.concat([test_clean['building_id'], pd.DataFrame(pd.DataFrame(model.predict(submis_x), columns = [1,2,3]).idxmax(axis = 1), columns = ['damage_grade'])], axis = 1).set_index('building_id')

# submis_df.to_csv('submis1.csv')

In [0]:
submis_df2 = pd.merge(left = pd.read_csv(test_vals)[['building_id']], right = submis_df, how = 'left', on = 'building_id').set_index('building_id')
submis_df2.to_csv('submis4.csv')